In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
import dnn_model

In [1]:
class BatchNorm(nn.BatchNorm2d):
    def __init__(self, num_features, eps=1e-05, momentum=0.1, weight=True, bias=True):
        super().__init__(num_features, eps=eps, momentum=momentum)
        self.weight.data.fill_(1.0)
        self.bias.data.fill_(0.0)
        self.weight.requires_grad = weight
        self.bias.requires_grad = bias


class GhostBatchNorm(BatchNorm):
    def __init__(self, num_features, num_splits, **kw):
        super().__init__(num_features, **kw)
        self.num_splits = num_splits
        self.register_buffer('running_mean', torch.zeros(num_features * self.num_splits))
        self.register_buffer('running_var', torch.ones(num_features * self.num_splits))

    def train(self, mode=True):
        if (self.training is True) and (mode is False):  # lazily collate stats when we are going to use them
            self.running_mean = torch.mean(self.running_mean.view(self.num_splits, self.num_features), dim=0).repeat(
                self.num_splits)
            self.running_var = torch.mean(self.running_var.view(self.num_splits, self.num_features), dim=0).repeat(
                self.num_splits)
        return super().train(mode)

    def forward(self, input):
        N, C, H, W = input.shape
        if self.training or not self.track_running_stats:
            return F.batch_norm(
                input.view(-1, C * self.num_splits, H, W), self.running_mean, self.running_var,
                self.weight.repeat(self.num_splits), self.bias.repeat(self.num_splits),
                True, self.momentum, self.eps).view(N, C, H, W)
        else:
            return F.batch_norm(
                input, self.running_mean[:self.num_features], self.running_var[:self.num_features],
                self.weight, self.bias, False, self.momentum, self.eps)


NameError: ignored

In [3]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = dnn_model.Net1().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
           Dropout-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 10, 24, 24]           1,450
           Dropout-4           [-1, 10, 24, 24]               0
         MaxPool2d-5           [-1, 10, 12, 12]               0
       BatchNorm2d-6           [-1, 10, 12, 12]              20
            Conv2d-7           [-1, 10, 10, 10]             910
           Dropout-8           [-1, 10, 10, 10]               0
            Conv2d-9             [-1, 16, 8, 8]           1,456
          Dropout-10             [-1, 16, 8, 8]               0
      BatchNorm2d-11             [-1, 16, 8, 8]              32
           Conv2d-12             [-1, 16, 6, 6]           2,320
          Dropout-13             [-1, 16, 6, 6]               0
           Conv2d-14             [-1, 1

/content/dnn_model.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:
from dataloader import TrainSetLoader

batch_size = 128
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
loader = TrainSetLoader(kwargs, batch_size)
train_loader = loader.data_loader("train", "MNIST")
test_loader = loader.data_loader("test", "MNIST")


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [0]:
# Net1 - without L1 & L2, With Batch normalization
from train import train_model
from test import test_model

# Session 5 final model
model = dnn_model.Net1().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 25):
    train_model(model, device, train_loader, optimizer, epoch)
    test_model(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/content/dnn_model.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.2893454134464264 batch_id=468 Accuracy=72.09: 100%|██████████| 469/469 [01:16<00:00,  6.15it/s]
loss=0.13874687254428864 batch_id=0 Accuracy=93.75:   0%|          | 1/469 [00:00<01:18,  5.94it/s]


Test set: Average loss: 0.1180, Accuracy: 9671/10000 (96.71%)



loss=0.12450223416090012 batch_id=468 Accuracy=96.24: 100%|██████████| 469/469 [01:15<00:00,  6.22it/s]
loss=0.04092518240213394 batch_id=0 Accuracy=99.22:   0%|          | 1/469 [00:00<01:16,  6.14it/s]


Test set: Average loss: 0.0664, Accuracy: 9787/10000 (97.87%)



loss=0.02072138525545597 batch_id=468 Accuracy=97.47: 100%|██████████| 469/469 [01:16<00:00,  6.15it/s]
loss=0.10791319608688354 batch_id=0 Accuracy=98.44:   0%|          | 1/469 [00:00<01:16,  6.11it/s]

...


Test set: Average loss: 0.0234, Accuracy: 9927/10000 (99.27%)



loss=0.04968872293829918 batch_id=468 Accuracy=99.03: 100%|██████████| 469/469 [01:16<00:00,  6.16it/s]
loss=0.024733087047934532 batch_id=0 Accuracy=99.22:   0%|          | 1/469 [00:00<01:20,  5.78it/s]


Test set: Average loss: 0.0229, Accuracy: 9916/10000 (99.16%)



loss=0.022303065285086632 batch_id=468 Accuracy=99.06: 100%|██████████| 469/469 [01:16<00:00,  6.15it/s]



Test set: Average loss: 0.0239, Accuracy: 9931/10000 (99.31%)



In [0]:
# Net1 - with L2, with BN
from train import train_model
from test import test_model

# Session 5 final model
model = dnn_model.Net1().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0, dampening=0, weight_decay=0, nesterov=False)

for epoch in range(1, 25):
    train_model(model, device, train_loader, optimizer, epoch)
    test_model(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/content/dnn_model.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=2.159263849258423 batch_id=468 Accuracy=17.53: 100%|██████████| 469/469 [01:22<00:00,  5.71it/s]
loss=2.106940507888794 batch_id=0 Accuracy=19.53:   0%|          | 1/469 [00:00<01:20,  5.80it/s]


Test set: Average loss: 2.1149, Accuracy: 2606/10000 (26.06%)



loss=0.9036510586738586 batch_id=468 Accuracy=47.20: 100%|██████████| 469/469 [01:21<00:00,  5.74it/s]
loss=0.8817183375358582 batch_id=0 Accuracy=74.22:   0%|          | 1/469 [00:00<01:20,  5.85it/s]


Test set: Average loss: 0.8776, Accuracy: 7538/10000 (75.38%)



loss=0.3488036096096039 batch_id=468 Accuracy=82.37: 100%|██████████| 469/469 [01:21<00:00,  5.77it/s]
loss=0.3291482925415039 batch_id=0 Accuracy=90.62:   0%|          | 1/469 [00:00<01:22,  5.64it/s]

...


Test set: Average loss: 0.0409, Accuracy: 9869/10000 (98.69%)



loss=0.15186090767383575 batch_id=468 Accuracy=98.07: 100%|██████████| 469/469 [01:21<00:00,  5.75it/s]
loss=0.046191733330488205 batch_id=0 Accuracy=97.66:   0%|          | 1/469 [00:00<01:19,  5.92it/s]


Test set: Average loss: 0.0425, Accuracy: 9869/10000 (98.69%)



loss=0.04372880980372429 batch_id=468 Accuracy=98.15: 100%|██████████| 469/469 [01:20<00:00,  5.80it/s]



Test set: Average loss: 0.0385, Accuracy: 9882/10000 (98.82%)



In [5]:
# Net1 - with L2, with BN - weight_decay=5e-4
from train import train_model
from test import test_model

# Session 5 final model
model = dnn_model.Net1().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0, dampening=0, weight_decay=5e-4, nesterov=False)

for epoch in range(1, 25):
    train_model(model, device, train_loader, optimizer, epoch)
    test_model(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/content/dnn_model.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=2.1726458072662354 batch_id=468 Accuracy=17.58: 100%|██████████| 469/469 [00:24<00:00, 19.24it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.1188, Accuracy: 2635/10000 (26.35%)



loss=0.9015507698059082 batch_id=468 Accuracy=47.34: 100%|██████████| 469/469 [00:24<00:00, 18.99it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.8719, Accuracy: 7456/10000 (74.56%)



loss=0.40407347679138184 batch_id=468 Accuracy=82.74: 100%|██████████| 469/469 [00:24<00:00, 19.44it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0441, Accuracy: 9870/10000 (98.70%)



loss=0.04532359912991524 batch_id=468 Accuracy=98.13: 100%|██████████| 469/469 [00:24<00:00, 19.34it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0444, Accuracy: 9861/10000 (98.61%)



loss=0.0793975219130516 batch_id=468 Accuracy=98.15: 100%|██████████| 469/469 [00:24<00:00, 19.43it/s]



Test set: Average loss: 0.0423, Accuracy: 9877/10000 (98.77%)



In [0]:
# Net1 - with L1, With Batch normalization
from train import train_model
from test import test_model

# Session 5 final model
model = dnn_model.Net1().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train_model(model, device, train_loader, optimizer, epoch, "L1")
    test_model(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/content/dnn_model.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.5337293148040771 batch_id=468 Accuracy=71.70: 100%|██████████| 469/469 [01:25<00:00,  5.48it/s]



Test set: Average loss: 0.1589, Accuracy: 9566/10000 (95.66%)



In [0]:
# Net1 - with L1, With Batch normalization
from train import train_model
from test import test_model

# Session 5 final model
model = dnn_model.Net1().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 25):
    train_model(model, device, train_loader, optimizer, epoch, "L1")
    test_model(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/content/dnn_model.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.5337293148040771 batch_id=468 Accuracy=71.70: 100%|██████████| 469/469 [01:25<00:00,  5.50it/s]
loss=0.3812520503997803 batch_id=0 Accuracy=91.41:   0%|          | 1/469 [00:00<01:25,  5.50it/s]


Test set: Average loss: 0.1589, Accuracy: 9566/10000 (95.66%)



loss=0.29899686574935913 batch_id=468 Accuracy=96.16: 100%|██████████| 469/469 [01:26<00:00,  5.44it/s]
loss=0.28444990515708923 batch_id=0 Accuracy=99.22:   0%|          | 1/469 [00:00<01:21,  5.74it/s]


Test set: Average loss: 0.0839, Accuracy: 9746/10000 (97.46%)



loss=0.23820088803768158 batch_id=468 Accuracy=97.06: 100%|██████████| 469/469 [01:26<00:00,  5.42it/s]
loss=0.2842409312725067 batch_id=0 Accuracy=96.88:   0%|          | 1/469 [00:00<01:22,  5.66it/s]

...


Test set: Average loss: 0.0544, Accuracy: 9835/10000 (98.35%)



loss=0.25099697709083557 batch_id=468 Accuracy=98.06: 100%|██████████| 469/469 [01:25<00:00,  5.52it/s]
loss=0.19664454460144043 batch_id=0 Accuracy=99.22:   0%|          | 1/469 [00:00<01:21,  5.72it/s]


Test set: Average loss: 0.0674, Accuracy: 9797/10000 (97.97%)



loss=0.16954964399337769 batch_id=468 Accuracy=98.09: 100%|██████████| 469/469 [01:24<00:00,  5.52it/s]



Test set: Average loss: 0.0477, Accuracy: 9856/10000 (98.56%)



In [0]:
# Net1 - with L1 & L2, with BN
from train import train_model
from test import test_model

# Session 5 final model
model = dnn_model.Net1().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0, dampening=0, weight_decay=0, nesterov=False)

for epoch in range(1, 25):
    train_model(model, device, train_loader, optimizer, epoch, 'L1')
    test_model(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/content/dnn_model.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=2.392559289932251 batch_id=468 Accuracy=17.49: 100%|██████████| 469/469 [01:25<00:00,  5.50it/s]
loss=2.349015951156616 batch_id=0 Accuracy=18.75:   0%|          | 1/469 [00:00<01:25,  5.46it/s]


Test set: Average loss: 2.1371, Accuracy: 2594/10000 (25.94%)



loss=1.2016850709915161 batch_id=468 Accuracy=45.18: 100%|██████████| 469/469 [01:25<00:00,  5.51it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.9352, Accuracy: 7383/10000 (73.83%)



loss=0.5858258008956909 batch_id=468 Accuracy=81.87: 100%|██████████| 469/469 [01:25<00:00,  5.48it/s]
loss=0.5638421177864075 batch_id=0 Accuracy=90.62:   0%|          | 1/469 [00:00<01:23,  5.57it/s]

...


Test set: Average loss: 0.0481, Accuracy: 9865/10000 (98.65%)



loss=0.3000286817550659 batch_id=468 Accuracy=98.06: 100%|██████████| 469/469 [01:15<00:00,  6.20it/s]
loss=0.2138294279575348 batch_id=0 Accuracy=98.44:   0%|          | 1/469 [00:00<01:12,  6.43it/s]


Test set: Average loss: 0.0488, Accuracy: 9883/10000 (98.83%)



loss=0.23853173851966858 batch_id=468 Accuracy=98.05: 100%|██████████| 469/469 [01:15<00:00,  6.17it/s]



Test set: Average loss: 0.0469, Accuracy: 9870/10000 (98.70%)



In [6]:
# Net1 - with L1 & L2, with BN
from train import train_model
from test import test_model

# Session 5 final model
model = dnn_model.Net1().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0, dampening=0, weight_decay=5e-3, nesterov=False)

for epoch in range(1, 25):
    train_model(model, device, train_loader, optimizer, epoch, 'L1')
    test_model(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/content/dnn_model.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=2.060058832168579 batch_id=468 Accuracy=23.82: 100%|██████████| 469/469 [00:25<00:00, 18.16it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.9571, Accuracy: 3135/10000 (31.35%)



loss=0.943038284778595 batch_id=468 Accuracy=57.77: 100%|██████████| 469/469 [00:25<00:00, 18.44it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.8292, Accuracy: 7565/10000 (75.65%)



loss=0.6774700880050659 batch_id=468 Accuracy=81.72: 100%|██████████| 469/469 [00:25<00:00, 18.39it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0947, Accuracy: 9773/10000 (97.73%)



loss=0.2191462218761444 batch_id=468 Accuracy=97.44: 100%|██████████| 469/469 [00:25<00:00, 18.16it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0793, Accuracy: 9816/10000 (98.16%)



loss=0.27482134103775024 batch_id=468 Accuracy=97.43: 100%|██████████| 469/469 [00:25<00:00, 18.27it/s]



Test set: Average loss: 0.0759, Accuracy: 9811/10000 (98.11%)

